* https://spark.apache.org/docs/2.1.0/ml-classification-regression.html
* https://spark.apache.org/docs/latest/mllib-collaborative-filtering.html
* https://spark.apache.org/docs/2.2.0/ml-collaborative-filtering.html

In [18]:
from pyspark import SparkConf, SparkContext


from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import LinearRegression

from pyspark.ml.evaluation import RegressionEvaluator
#from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from pyspark.sql import Row

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS


import re

In [2]:
# set up environment
conf = SparkConf() \
    .setAppName("MovieLensALS") \
    .set("spark.executor.memory", "2g")
sc = SparkContext(conf=conf)

In [14]:
path_data = "/home/p5hngk/Downloads/GitHub/SD_701---Data_Mining/ml-latest-small"

# Load and parse the ratings data
data = sc.textFile(path_data+"/ratings.csv").map(lambda x: x.split(","))
ratings = data.filter(lambda x: x[0]!="userId")
ratings.take(5)

[['1', '1', '4.0', '964982703'],
 ['1', '3', '4.0', '964981247'],
 ['1', '6', '4.0', '964982224'],
 ['1', '47', '5.0', '964983815'],
 ['1', '50', '5.0', '964982931']]

In [25]:
(training, test) = ratings.randomSplit([0.8, 0.2])
# Training the model

evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

model = evaluator.evaluate(training)


AttributeError: 'PipelinedRDD' object has no attribute '_jdf'

In [10]:
(training, test) = ratings.randomSplit([0.8, 0.2])

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)

TypeError: ALS() takes no arguments

In [ ]:
# Load and parse the movies data
data2 = sc.textFile(path_data+"/movies.csv")
movies = data2.map(parseCSV) #.filter(lambda x : x[0]!="movieId") #on enlève la première ligne
movies.take(5)

In [ ]:
data2.take(5)

In [ ]:
ratings.map(lambda x: (x[0], x[1])).take(5)

In [ ]:
## https://spark.apache.org/docs/2.2.0/ml-collaborative-filtering.html

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(training)

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

# We can also use the multinomial family for binary classification
mlr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, family="multinomial")

# Fit the model
mlrModel = mlr.fit(training)

# Print the coefficients and intercepts for logistic regression with multinomial family
print("Multinomial coefficients: " + str(mlrModel.coefficientMatrix))
print("Multinomial intercepts: " + str(mlrModel.interceptVector))

In [ ]:
from pyspark.mllib.recommendation import ALS, Rating
from pyspark.mllib.evaluation import RegressionMetrics, RankingMetrics

# Read in the ratings data
lines = sc.textFile("data/mllib/sample_movielens_data.txt")

def parseLine(line):
    fields = line.split("::")
    return Rating(int(fields[0]), int(fields[1]), float(fields[2]) - 2.5)
ratings = lines.map(lambda r: parseLine(r))

# Train a model on to predict user-product ratings
model = ALS.train(ratings, 10, 10, 0.01)

# Get predicted ratings on all existing user-product pairs
testData = ratings.map(lambda p: (p.user, p.product))
predictions = model.predictAll(testData).map(lambda r: ((r.user, r.product), r.rating))

ratingsTuple = ratings.map(lambda r: ((r.user, r.product), r.rating))
scoreAndLabels = predictions.join(ratingsTuple).map(lambda tup: tup[1])

# Instantiate regression metrics to compare predicted and actual ratings
metrics = RegressionMetrics(scoreAndLabels)

# Root mean squared error
print("RMSE = %s" % metrics.rootMeanSquaredError)

# R-squared
print("R-squared = %s" % metrics.r2)